In [10]:
import pyspark
import os
import shutil
root_folder = "/home/trungdc/unimelb/MAST30024/asm/project 1/"
data_dir = os.path.join(root_folder, "Data/Trip")


import pyspark.sql.functions as F

from pyspark.sql.functions import col
from pyspark.sql.types import *

In [2]:
from pyspark.sql import SparkSession
import warnings
warnings.filterwarnings("ignore")
spark = SparkSession.builder.getOrCreate()
spark.conf.set('spark.sql.execution.arrow.pyspark.enabled', True)

# Download yellow trip record (Month: 2,4,6,8,10,12) to folder: Data/Yellow

In [3]:
sdf = spark.read.format("csv").option("header", "true").load(os.path.join(data_dir,"Yellow","*.csv"))

In [14]:
spark.conf.set('spark.sql.execution.arrow.pyspark.enabled', True)
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
sdf.createOrReplaceTempView("trip")
sdf.limit(5)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
1,2018-02-01 00:01:58,2018-02-01 00:04:03,1,.00,1,N,145,145,2,3.5,0.5,0.5,0,0,0.3,4.8
1,2018-02-01 00:56:48,2018-02-01 00:57:42,1,2.90,1,N,145,145,2,2.5,0.5,0.5,0,0,0.3,3.8
1,2018-02-01 00:04:42,2018-02-01 00:19:32,1,5.80,1,N,236,119,2,18.5,0.5,0.5,0,0,0.3,19.8
1,2018-02-01 00:38:10,2018-02-01 00:40:16,1,.30,1,N,82,82,2,3.5,0.5,0.5,0,0,0.3,4.8
1,2018-02-01 00:43:03,2018-02-01 00:59:26,2,2.60,1,N,82,7,1,13,0.5,0.5,1,0,0.3,15.3


In [11]:
print((sdf.count(), len(sdf.columns)))

(92931995, 17)


In [15]:
sql_query = """
SELECT *
FROM trip
ORDER BY "tpep_pickup_datetime"
"""
out = spark.sql(sql_query)
out.head()

Row(VendorID='1', tpep_pickup_datetime='2018-02-01 00:01:58', tpep_dropoff_datetime='2018-02-01 00:04:03', passenger_count='1', trip_distance='.00', RatecodeID='1', store_and_fwd_flag='N', PULocationID='145', DOLocationID='145', payment_type='2', fare_amount='3.5', extra='0.5', mta_tax='0.5', tip_amount='0', tolls_amount='0', improvement_surcharge='0.3', total_amount='4.8')

In [16]:
out.repartition(1).write.csv(os.path.join(data_dir,"yellow.csv"), header=True)

In [11]:
# reorder foldter and file
for file in os.listdir(os.path.join(data_dir, "yellow.csv")):
    if file.endswith(".csv"):
        os.rename(os.path.join(data_dir, "yellow.csv", file), os.path.join(data_dir, "yellow_concat.csv"))
shutil.rmtree(os.path.join(data_dir, "yellow.csv"))